# Data modification test

In [17]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [18]:
ALICE, BOB = 0, 1

[alice_files, bob_files] = fetch_data(subjects=[ALICE, BOB], recording=[1])

raw_train = mne.io.read_raw_edf(alice_files[0], stim_channel='Event marker',
                                misc=['Temp rectal'], preload=True)
annot_train = mne.read_annotations(alice_files[1])

raw_train.set_annotations(annot_train, emit_warning=False)

# raw_train.plot(start=60, duration=60,
#                scalings=dict(eeg=1e-4, resp=1e3, eog=1e-4, emg=1e-7,
#                              misc=1e-1))

Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from /home/anrath/mne_data/physionet-sleep-data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...


<RawEDF | SC4001E0-PSG.edf, 7 x 7950000 (79500.0 s), ~424.6 MB, data loaded>

In [34]:
# raw_copy = raw_train.copy()
# func_data = raw_copy.apply_function(np.abs)
# func_data.get_data()

In [40]:
from peak_finder import PeakFinder as pf

t = np.arange(0, 3, 0.01)
signal_sin = np.sin(np.pi*t) - np.sin(0.5*np.pi*t)
mne_sin_peak_locs, mne_sin_peak_mags = mne.preprocessing.peak_finder(signal_sin) 

signal_eeg = raw_train.get_data()[0]
format_percent = lambda x, y: np.round(len(x)/len(y), 4)
mne_eeg_peak_locs, mne_eeg_peak_mags = mne.preprocessing.peak_finder(raw_train.get_data()[0])

def success_metrics(results, signal='eeg', string=""):
    if signal == 'eeg':
        signal = signal_eeg
        mne_peak_locs = mne_eeg_peak_locs
    elif signal == 'sin':
        signal = signal_sin
        mne_peak_locs = mne_sin_peak_locs

    common_peaks = np.intersect1d(results, mne_peak_locs)
    common_peaks_len = len(common_peaks)

    results_len = len(results)
    peak_to_signal_ratio = format_percent(results, signal)

    actual_to_predicted_peak_count_ratio = format_percent(results, mne_peak_locs)

    print(string + f"Peaks: {results_len} ({peak_to_signal_ratio}), Intersect Num: {common_peaks_len} ({actual_to_predicted_peak_count_ratio})")
    return None


Found 29454 significant peaks


In [37]:
minimum_height = 4e-5
edges = ['rising', 'falling', 'both', None]
# mph = None
# print(f"mne peak number: {len(mne_peak_locs)}")
success_metrics(mne_eeg_peak_locs, signal='eeg', string="MNE: ")
for edge in edges:
    ind_eeg_peak_typing_finder = pf.peak_typing_finder(signal_eeg, minimum_height=minimum_height, minimum_distance=1, edge=edge)
    success_metrics(ind_eeg_peak_typing_finder, signal='eeg', string=f"Edge: {edge}, ")

print('\n')

success_metrics(mne_sin_peak_locs, signal='sin', string="MNE: ")
for edge in edges:
    ind_sin_peak_typing_finder = pf.peak_typing_finder(signal_sin, minimum_height=minimum_height, minimum_distance=1, edge=edge)
    success_metrics(ind_sin_peak_typing_finder, signal='sin', string=f"Edge: {edge}, ")

MNE: Peaks: 29454 (0.0037), Intersect Num: 29454 (1.0)
Edge: rising, Peaks: 161038 (0.0203), Intersect Num: 25484 (5.4674)
Edge: falling, Peaks: 161118 (0.0203), Intersect Num: 25323 (5.4702)
Edge: both, Peaks: 162401 (0.0204), Intersect Num: 25484 (5.5137)
Edge: None, Peaks: 159755 (0.0201), Intersect Num: 25323 (5.4239)


MNE: Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Edge: rising, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Edge: falling, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Edge: both, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Edge: None, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)


In [38]:
peaks_eeg = {}
success_metrics(mne_eeg_peak_locs, signal='eeg', string="MNE: ")
distances = [15, 35, 50, 100, 155]
for distance in distances:
    peaks_eeg[distance] = pf.naive_logical_find_peaks(signal_eeg, min_distance=distance)
    success_metrics(peaks_eeg[distance], signal='eeg', string=f"Distance: {distance}, ")

print('\n')

peaks_sin = {}
success_metrics(mne_sin_peak_locs, signal='sin', string="MNE: ")
for distance in distances:
    peaks_sin[distance] = pf.naive_logical_find_peaks(signal_sin, min_distance=distance)
    success_metrics(peaks_sin[distance], signal='sin', string=f"Distance: {distance}, ")

MNE: Peaks: 29454 (0.0037), Intersect Num: 29454 (1.0)
Distance: 15, Peaks: 188047 (0.0237), Intersect Num: 28906 (6.3844)
Distance: 35, Peaks: 87758 (0.011), Intersect Num: 27407 (2.9795)
Distance: 50, Peaks: 63595 (0.008), Intersect Num: 25730 (2.1591)
Distance: 100, Peaks: 34871 (0.0044), Intersect Num: 21150 (1.1839)
Distance: 155, Peaks: 26413 (0.0033), Intersect Num: 18193 (0.8968)


MNE: Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Distance: 15, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Distance: 35, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Distance: 50, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Distance: 100, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Distance: 155, Peaks: 2 (0.0067), Intersect Num: 2 (1.0)


In [39]:
success_metrics(mne_eeg_peak_locs, signal='eeg', string="MNE: ")
ind_eeg_naive_mathematical_find_peaks = pf.naive_mathematical_find_peaks(signal_eeg)
success_metrics(ind_eeg_naive_mathematical_find_peaks, signal='eeg')

print('\n')

success_metrics(mne_sin_peak_locs, signal='sin', string="MNE: ")
ind_sin_naive_mathematical_find_peaks = pf.naive_mathematical_find_peaks(signal_sin)
success_metrics(ind_sin_naive_mathematical_find_peaks, signal='sin')

MNE: Peaks: 29454 (0.0037), Intersect Num: 29454 (1.0)
Peaks: 2128087 (0.2677), Intersect Num: 29070 (72.2512)


MNE: Peaks: 2 (0.0067), Intersect Num: 2 (1.0)
Peaks: 2 (0.0067), Intersect Num: 1 (1.0)
